In [26]:
from torch.nn import Sequential, Module, ReLU, Conv2d, Linear, MaxPool2d, LogSoftmax, NLLLoss, Dropout, BatchNorm2d, LeakyReLU, GELU, SELU, Mish, CrossEntropyLoss
from torchvision import transforms, datasets, models
from torch.utils.data import DataLoader
from torch import flatten, float, no_grad
from torch.optim import Adam
import torch
import wandb
import math

In [27]:
def get_data(param, type):
    if(type.lower() == 'train'):
        transform = transforms.Compose([
            transforms.Resize(256),
            # transforms.RandomResizedCrop(size=256, scale=(0.8, 1.0)),
            transforms.RandomHorizontalFlip(p=0.3),
            transforms.RandomRotation(degrees=12),
            transforms.ColorJitter(),
            transforms.CenterCrop(224),
            transforms.ToTensor(), 
            transforms.Normalize(mean = [0.485, 0.456, 0.406], std = [0.229, 0.224, 0.225])  
        ])
        # print("hi")
        # print(param)
        tdataset = datasets.ImageFolder(root=param['train_data_dir'], transform=transform)
        total = len(tdataset)
        train_sample = math.ceil(total*(0.8))
        val_sample = total-train_sample
        # print(total, train_sample, val_sample)
        train_dataset, validation_dataset = torch.utils.data.random_split(tdataset, [train_sample, val_sample])
        train_dataloader = DataLoader(train_dataset, batch_size=param['batch_size'], shuffle=True)
        validation_dataloader = DataLoader(validation_dataset, batch_size=param['batch_size'], shuffle=False)
        return train_dataloader, validation_dataloader
    
    else:
        transform = transforms.Compose([
            transforms.RandomResizedCrop(size=256, scale=(0.8, 1.0)),
            transforms.CenterCrop(size=224),
            transforms.ToTensor(), 
            transforms.Normalize(mean = [0.485, 0.456, 0.406], std = [0.229, 0.224, 0.225])  
        ])
        test_dataset = datasets.ImageFolder(root=param['test_data_dir'], transform=transform)
        test_dataloader = DataLoader(test_dataset, batch_size=param['batch_size'])
        return test_dataloader

## Transfer Learning using VGG16 model

In [38]:
def train():

    wandb.init()
    param = wandb.config
    wandb.run.name = f'vgg16_strategy_{param.strategy}_batchSz_{param.batch_size}_epochs_{param.epochs}'

    # param = {
    #     "batch_size": 32,
    #     "epochs": 5,
    #     "train_data_dir": "./data/train",
    #     "test_data_dir": "./data/val"
    # }


    device = torch.device("mps" if torch.backends.mps.is_available() else "cpu")
    pmodel = models.googlenet(pretrained=True)

    num_features = pmodel.fc.in_features

    pmodel.fc = Linear(num_features,10)

    for name, par in pmodel.named_parameters():
        if name not in ['fc.weight', 'fc.bias']:
            par.requires_grad = False

    total_params = sum(p.numel() for p in pmodel.parameters())
    print(f'{total_params:,} total parameters.')
    total_trainable_params = sum(
        p.numel() for p in pmodel.parameters() if p.requires_grad)
    print(f'{total_trainable_params:,} training parameters.')

    # for x in pmodel.parameters():
    #     x.requires_grad = False
    # pmodel.fc.requires_grad = True

    
    
    pmodel = pmodel.to(device)
    optimizer = Adam(pmodel.parameters())
    loss_function = CrossEntropyLoss()
    train_data_loader, validation_data_loader = get_data(param, 'train')

    for epo in range(param['epochs']):
        totalTrainLoss = 0
        totalValLoss = 0
        trainCorrect = 0
        valCorrect = 0
        train_counter=0
        validation_counter=0
        pmodel.train()
        for (image, label) in train_data_loader:
            (image, label) = (image.to(device), label.to(device))
            prediction = pmodel(image)
            loss = loss_function(prediction, label)
            ## no optimize.zero_grad() ...
            loss.backward()
            optimizer.step()

            totalTrainLoss += loss
            trainCorrect += (prediction.argmax(1) == label).type(float).sum().item()
            train_counter+=1
            # print(train_counter)

        pmodel.eval()
        with no_grad():
            for (image, label) in validation_data_loader:
                (image, label) = (image.to(device), label.to(device))
                pred = pmodel(image)
                loss = loss_function(pred, label)
                totalValLoss += loss
                valCorrect += (pred.argmax(1) == label).type(float).sum().item()
                validation_counter += 1

        tr_ls = (totalTrainLoss/train_counter).cpu().detach().numpy()
        tr_acc = trainCorrect/len(train_data_loader.dataset)
        val_ls = (totalValLoss/validation_counter).cpu().detach().numpy()
        val_acc = valCorrect/len(validation_data_loader.dataset)
        print(f"Epoch --> {epo}")
        print(f"Train Loss --> {tr_ls}")
        print(f"Train Accuracy --> {tr_acc}")
        print(f"Validation Loss --> {val_ls}")
        print(f"Validation Accuracy --> {val_acc}")
        print("-----------------------------------------------------------")
        
        lg={
            'epoch': epo+1,
            'tr_accuracy': tr_acc,
            'val_accuracy': val_acc,
            'tr_loss': tr_ls,
            'val_loss': val_ls
        }
        wandb.log(lg)

    # torch.save(model, checkpoint_path)

    

In [39]:
# train()

In [40]:
sweep_config = {
  "method": "grid",  # Use grid search for this example
  "name": "PartB VGG16 Sweep",
  "metric": {"goal": "maximize", "name": "val_accuracy"},
  "parameters": {
    "batch_size":{"values": [32]},
    "epochs":{"values": [5]},
    "strategy":{"values": ['all_freeze']},  ## K freeze, No Freeze
    "train_data_dir":{"values": ["./data/train"]},
    "test_data_dir":{"values": ["./data/val"]}
  }
}

In [41]:
# wandb.init()
sweep_id = wandb.sweep(sweep_config, project="cs6910_assignment2")

Failed to detect the name of this notebook, you can set it manually with the WANDB_NOTEBOOK_NAME environment variable to enable code saving.


Create sweep with ID: 4eidwpsk
Sweep URL: https://wandb.ai/cs23m070/cs6910_assignment2/sweeps/4eidwpsk


In [42]:
wandb.agent(sweep_id, function=train, count=1)
wandb.finish()

wandb: Agent Starting Run: epph7rw6 with config:
wandb: 	batch_size: 32
wandb: 	dense_neurons: 128
wandb: 	dropout: 0.3
wandb: 	epochs: 5
wandb: 	strategy: all_freeze
wandb: 	test_data_dir: ./data/val
wandb: 	train_data_dir: ./data/train
Failed to detect the name of this notebook, you can set it manually with the WANDB_NOTEBOOK_NAME environment variable to enable code saving.
wandb: Currently logged in as: cs23m070. Use `wandb login --relogin` to force relogin


/Users/tirth/anaconda3/envs/dl/lib/python3.10/site-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/Users/tirth/anaconda3/envs/dl/lib/python3.10/site-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=GoogLeNet_Weights.IMAGENET1K_V1`. You can also use `weights=GoogLeNet_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


5,610,154 total parameters.
10,250 training parameters.
Epoch --> 0
Train Loss --> 10.22955322265625
Train Accuracy --> 0.271375
Validation Loss --> 10.036049842834473
Validation Accuracy --> 0.3456728364182091
-----------------------------------------------------------
Epoch --> 1
Train Loss --> 22.519357681274414
Train Accuracy --> 0.326375
Validation Loss --> 22.084354400634766
Validation Accuracy --> 0.3566783391695848
-----------------------------------------------------------
Epoch --> 2
Train Loss --> 24.801694869995117
Train Accuracy --> 0.319875
Validation Loss --> 20.846763610839844
Validation Accuracy --> 0.4112056028014007
-----------------------------------------------------------
Epoch --> 3
Train Loss --> 22.853111267089844
Train Accuracy --> 0.477375
Validation Loss --> 22.256725311279297
Validation Accuracy --> 0.5147573786893447
-----------------------------------------------------------
Epoch --> 4
Train Loss --> 24.960655212402344
Train Accuracy --> 0.495
Validation

epoch,▁▃▅▆█
tr_accuracy,▁▃▃▇█
tr_loss,▁▇█▇█
val_accuracy,▁▁▃▇█
val_loss,▁█▇█▇
epoch,5
tr_accuracy,0.495
tr_loss,24.96066
val_accuracy,0.53277
val_loss,20.42535
